In [1]:
import pandas as pd
import numpy as np

In [2]:
DF=pd.read_csv('ShitCoinsv2.csv')
DF=DF.drop('Unnamed: 0',axis=1)
DF=DF.reindex(sorted(DF.columns), axis=1)
#alphabetically sorted
DF

,announcement,category,chat,explorer,facebook,is_fiat,market_cap_change,message_board,name,rate_close,rate_high,rate_open,reddit,source_code,symbol,technical_doc,twitter,website
0,0,1,1,1,0,0,-0.979357,1,0,2.331322,2.339022,2.331244,0,0,0,0,1,1
1,0,1,1,1,0,0,-2.949193,1,0,2.331322,2.339022,2.331244,0,0,0,0,1,1
2,0,1,1,1,0,0,-0.560834,1,0,2.331322,2.339022,2.331244,0,0,0,0,1,1
3,0,1,1,1,0,0,-2.013200,1,0,2.331322,2.339022,2.331244,0,0,0,0,1,1
4,1,0,1,3,0,0,-0.979357,0,3,2.331322,2.339022,2.331244,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,0,0,0,1,0,0,-0.560834,0,46,0.007962,0.007974,0.007958,0,0,53,0,0,1
352,1,1,2,1,0,0,-0.231262,1,117,0.000581,0.000591,0.000582,0,1,54,1,1,2
353,1,0,0,0,0,0,-0.231262,0,90,0.000581,0.000591,0.000582,0,0,54,0,0,1
354,0,0,1,2,0,0,-0.231262,0,116,0.000581,0.000591,0.000582,0,0,54,0,1,1


In [3]:
#training the model to classify tokens and coins apparts
y=DF['category']
x=DF.drop('category',axis=1)
#nheydo Y men X
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=30)

In [4]:
from sklearn.linear_model import LogisticRegression
modelDf=LogisticRegression()
modelDf.fit(x_train,y_train)

C:\Users\warda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [5]:
from sklearn.metrics import *
y_predict=modelDf.predict(x_test)
print("accuracy_score: ", accuracy_score(y_test,y_predict))

accuracy_score:  0.875


In [39]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.64      0.58      0.61        12
           1       0.92      0.93      0.93        60

    accuracy                           0.88        72
   macro avg       0.78      0.76      0.77        72
weighted avg       0.87      0.88      0.87        72



In [6]:
# E = [ 51.,  20., 1.06716151, 3.07469562, 4.06646845, -398.73589524, 1.]
# modelDf.predict([E])
# #eg 
# ctrlA ctrl/ >to comment all the lines

In [17]:
#Deployment

#create a function to predict given data with a model
c = "announcement"
code = [f"""
<div>
\t<label for="{c}">{c}</label>
\t<input type='number' name='{c}' value=''>
</div>
""" for c in x.columns]

code = "\n".join(code)

print(code)


<div>
	<label for="announcement">announcement</label>
	<input type='number' name='announcement' value=''>
</div>


<div>
	<label for="chat">chat</label>
	<input type='number' name='chat' value=''>
</div>


<div>
	<label for="explorer">explorer</label>
	<input type='number' name='explorer' value=''>
</div>


<div>
	<label for="facebook">facebook</label>
	<input type='number' name='facebook' value=''>
</div>


<div>
	<label for="is_fiat">is_fiat</label>
	<input type='number' name='is_fiat' value=''>
</div>


<div>
	<label for="market_cap_change">market_cap_change</label>
	<input type='number' name='market_cap_change' value=''>
</div>


<div>
	<label for="message_board">message_board</label>
	<input type='number' name='message_board' value=''>
</div>


<div>
	<label for="name">name</label>
	<input type='number' name='name' value=''>
</div>


<div>
	<label for="rate_close">rate_close</label>
	<input type='number' name='rate_close' value=''>
</div>


<div>
	<label for="rate_high">rate_high

In [36]:
import pickle 
f=open('encoder.pkl', 'rb')
encoder=pickle.load(f)
f.close()
#rb<<<read in binary

In [37]:
from flask import Flask, request
from flask.templating import render_template
app = Flask('app',template_folder='templates')


@app.route('/', methods=["GET","POST"])
def predict():
    if request.method=="GET":
        return render_template("index.html",columnz=x.columns)
    elif request.method=="POST":
        D=dict(request.form)
        D=list(D.values())
        pred=modelDf.predict([D])
        pred=encoder.inverse_transform(pred)
        return {
            "category":pred.tolist()
        }
    #else not necessary 
    

app.run()

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jan/2023 17:25:56] "GET / HTTP/1.1" 200 -
C:\Users\warda\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\warda\anaconda3\lib\site-packages\sklearn\base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)
127.0.0.1 - - [12/Jan/2023 17:25:58] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2023 17:26:07] "GET / HTTP/1.1" 200 -
C:\Users\warda\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\warda\anaconda3\lib\site-packages\skle